# Multivariate LSTM-Based Energy Consumption Forecasting and Anomaly Detection

## **Aim**
The aim of this project is to develop a machine learning system that:
- Accurately forecasts energy consumption in smart grids using historical usage and environmental data.
- Detects anomalies by comparing real-time consumption data with model predictions.
- Provides actionable insights in real-time through an interactive dashboard for energy managers and stakeholders.

## **Objectives**
1. Collect and preprocess multivariate time-series energy usage data.
2. Train a Long Short-Term Memory (LSTM) model to predict short-term and medium-term energy consumption.
3. Implement anomaly detection logic to flag abnormal consumption patterns.
4. Visualize forecasts and anomalies interactively using Streamlit.
5. Support real-time monitoring to enable preventive measures and energy optimization.

## **Technologies**
- **Python**: Core programming language.
- **TensorFlow / Keras**: For building and training the LSTM model.
- **Streamlit**: For dashboard development and interactivity.
- **Pandas / NumPy / Scikit-learn**: For data preprocessing.
- **Matplotlib / Plotly**: For visualizations.
- **Kaggle Energy Datasets**: For historical and environmental data.

# Step 2 – Data Acquisition & Preprocessing

Acquire multivariate energy consumption dataset from Kaggle or other sources.  
Clean, transform, and normalize data; create time-based features and split into training, validation, and test sets.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv("/content/AEP_hourly.csv", parse_dates=['Datetime'])

# Preview data
print("First rows:\n", df.head())
print("\nInfo:")
print(df.info())

# Check and handle missing values
print("\nMissing values per column:\n", df.isnull().sum())
df = df.dropna()  # Drop rows with missing values (or impute if needed)

# Feature engineering: extract time-based features
df['hour'] = df['Datetime'].dt.hour
df['dayofweek'] = df['Datetime'].dt.dayofweek
df['month'] = df['Datetime'].dt.month

# Normalize features except timestamp
feature_cols = ['AEP_MW', 'hour', 'dayofweek', 'month']
scaler = MinMaxScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

# Train / validation / test split
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)

train_df = df.iloc[:train_size]
val_df = df.iloc[train_size:train_size + val_size]
test_df = df.iloc[train_size + val_size:]

print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

The dataset contains 121,273 hourly records of energy consumption (AEP_MW) from 2004 to 2018, with no missing values.
After preprocessing and feature extraction, it was split into train (70%), validation (15%), and test (15%) sets.

# Step 3 – Model Development (LSTM)

Prepare sequential data for LSTM by creating sliding windows of past observations as input and the next time step as output.  
Build and train a multivariate LSTM model to forecast future energy consumption values.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Sequence preparation function
def create_sequences(data, target_column, window_size):
    """
    Creates input-output sequences for LSTM.
    data: DataFrame containing features
    target_column: column name for prediction target
    window_size: number of past time steps to look back
    """
    X, y = [], []
    for i in range(len(data) - window_size):
        # Select window_size rows of features
        X.append(data.iloc[i:i+window_size].values)
        # Select target value at the next time step
        y.append(data.iloc[i+window_size][target_column])
    return np.array(X), np.array(y)

# Parameters
window_size = 24  # 24 hours of past data as input
target_col = 'AEP_MW'

# Prepare sequences for train/val/test
X_train, y_train = create_sequences(train_df[['AEP_MW', 'hour', 'dayofweek', 'month']], target_col, window_size)
X_val, y_val = create_sequences(val_df[['AEP_MW', 'hour', 'dayofweek', 'month']], target_col, window_size)
X_test, y_test = create_sequences(test_df[['AEP_MW', 'hour', 'dayofweek', 'month']], target_col, window_size)

print(f"Train sequences shape: {X_train.shape} -> (samples, timesteps, features)")
print(f"Validation sequences shape: {X_val.shape}")
print(f"Test sequences shape: {X_test.shape}")

# LSTM Model Architecture
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(window_size, X_train.shape[2])))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(1))      # Output layer predicts a single future value

# Compile Model
model.compile(optimizer='adam', loss='mse')

# Train Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,            # Adjust as needed
    batch_size=32,
    verbose=1
)

# Evaluate Model
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {test_loss}")

LSTM model trained on 24-hour input windows achieved steady loss reduction over 10 epochs.  
Final test Mean Squared Error reached **2.71×10⁻⁴**, indicating high accuracy in forecasting hourly energy consumption.

# Step 4 – Forecasting & Anomaly Detection

Use the trained LSTM model to generate energy consumption forecasts on new data.  
Compare actual values with predictions to detect anomalies where prediction errors exceed a predefined threshold, enabling early identification of irregular energy usage patterns.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Calculate absolute errors
errors = np.abs(y_test - y_pred.flatten())

# Define anomaly detection threshold (e.g., mean + 3 standard deviations of training errors)
train_pred = model.predict(X_train)
train_errors = np.abs(y_train - train_pred.flatten())
threshold = train_errors.mean() + 3 * train_errors.std()

# Detect anomalies where error exceeds the threshold
anomalies = errors > threshold

print(f"Anomaly detection threshold: {threshold:.5f}")
print(f"Number of anomalies detected: {np.sum(anomalies)}")

# Plot some examples of actual vs predicted with anomalies highlighted
plt.figure(figsize=(15,5))
plt.plot(y_test[:500], label='Actual')
plt.plot(y_pred[:500], label='Predicted')
plt.scatter(np.where(anomalies[:500])[0], y_test[:500][anomalies[:500]], color='red', label='Anomaly', marker='x')
plt.title('Forecast vs Actual with Anomalies Highlighted (Sample)')
plt.xlabel('Time step')
plt.ylabel('Normalized Energy Consumption')
plt.legend()
plt.show()

Anomaly detection process identified **263** instances where prediction errors exceeded the threshold of **0.04937**, indicating unusual deviations between actual and forecasted energy consumption values.

# Step 5 – Visualization with Streamlit

Implement an interactive Streamlit dashboard to visualize historical energy consumption, LSTM forecasts, and detected anomalies.  
Enable users to explore time series data dynamically, view forecast accuracy, and identify anomaly events clearly for actionable insights.

In [ ]:
import pickle
with open("vis_df.pkl", "wb") as f:
    pickle.dump(vis_df, f)

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming previous variables: df, y_test, y_pred, anomalies, window_size, scaler

# Prepare a DataFrame for visualization
# Align prediction results with original timestamps for the test set
test_df_reset = test_df.reset_index(drop=True)
# Due to window_size offset, slice the test_df accordingly for alignment with predictions
vis_df = test_df_reset.iloc[window_size:].copy()
vis_df['Actual'] = y_test
vis_df['Predicted'] = y_pred.flatten()
vis_df['Anomaly'] = anomalies

# Inverse transform AEP_MW feature for actual and predicted values to original scale if needed
# Here only showing normalized values, can invert if scaler fitted with original scale
# Uncomment below if wanting original scale
# vis_df['Actual'] = scaler.inverse_transform(vis_df[['AEP_MW']].values)
# vis_df['Predicted'] = scaler.inverse_transform(vis_df[['Predicted']].values.reshape(-1,1))

# Streamlit app
st.title("Energy Consumption Forecasting and Anomaly Detection")

st.write("""
Interactive visualization of historical energy usage, predicted consumption, and detected anomalies.
Zoom in on specific time frames and observe consumption dynamics with highlights on anomalies.
""")

# Select number of points to display
num_points = st.slider("Select number of data points to display", min_value=100, max_value=len(vis_df), value=500)

plot_data = vis_df.head(num_points)

# Plotting
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(plot_data['Datetime'], plot_data['Actual'], label='Actual')
ax.plot(plot_data['Datetime'], plot_data['Predicted'], label='Predicted')

# Highlight anomalies
anomaly_points = plot_data[plot_data['Anomaly']]

ax.scatter(anomaly_points['Datetime'], anomaly_points['Actual'], color='red', label='Anomalies', marker='x')

ax.set_xlabel("Datetime")
ax.set_ylabel("Normalized Energy Consumption")
ax.set_title("Energy Consumption Forecast vs Actual with Anomalies")
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()

st.pyplot(fig)

In [ ]:
!pip install streamlit pyngrok

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("31Dxli0Ud8ZesMryaeVOZh0aCuX_33hN9DVQbL8faiX5zazqh")

In [ ]:
%%writefile app.py
import streamlit as st
st.title("Energy Consumption Forecasting and Anomaly Detection")
st.write("App content goes here...")

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# Title & description
st.title("Energy Consumption Forecasting and Anomaly Detection")
st.write("""
Interactive dashboard showcasing historical hourly energy usage,
LSTM-based forecasts, and detected anomalies.

Enables visual comparison of actual vs. predicted demand patterns
and highlights unusual consumption events for timely insights.
""")

# Example: load prepared visualization data (replace with actual vis_df DataFrame)
# Expecting columns: Datetime, Actual, Predicted, Anomaly
import pickle
with open("vis_df.pkl", "rb") as f:
    vis_df = pickle.load(f)

# Slider to choose number of points
num_points = st.slider("Select number of data points", 100, len(vis_df), 500)
plot_data = vis_df.head(num_points)

# Plot data
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(plot_data['Datetime'], plot_data['Actual'], label='Actual')
ax.plot(plot_data['Datetime'], plot_data['Predicted'], label='Predicted')
anomaly_points = plot_data[plot_data['Anomaly']]
ax.scatter(anomaly_points['Datetime'], anomaly_points['Actual'], color='red', label='Anomalies', marker='x')

ax.set_xlabel("Datetime")
ax.set_ylabel("Normalized Energy Consumption")
ax.set_title("Energy Consumption Forecast vs Actual with Anomalies")
ax.legend()
plt.xticks(rotation=45)
st.pyplot(fig)

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(public_url)

The live Streamlit dashboard is now running successfully at the ngrok URL.  
It shows the title, description, and an interactive slider ("Select number of data points") that controls how many points are displayed in the actual vs. predicted energy consumption graph.  
The graph includes red markers highlighting anomalies, enabling clear visual comparison between normal and unusual consumption patterns.

# Step 6 – Model Evaluation & Export

Assess forecasting accuracy using metrics such as RMSE, MAE, and R².  
Export the trained LSTM model and anomaly detection results for future use or integration into other systems.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Forecasts from your model: y_pred, actual: y_test
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"R²: {r2:.4f}")

In [ ]:
model.save("energy_forecast_model.h5")
print("✅ Model saved as energy_forecast_model.h5")

In [ ]:
import pandas as pd

anomalies_df = vis_df.copy()
anomalies_df.to_csv("anomalies_data.csv", index=False)
print("✅ Anomaly data saved as anomalies_data.csv")

In [ ]:
pip install tensorflow

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

loaded_model = load_model("energy_forecast_model.h5", custom_objects={"mse": MeanSquaredError()})

In [ ]:
# Evaluate the loaded model on the test data
results = loaded_model.evaluate(X_test, y_test, verbose=1)

# If results is a float, just print it directly
if isinstance(results, float):
    print(f"Test Loss: {results}")
else:
    # If multiple values (list/tuple), print them all
    print(f"Test Loss: {results[0]}")
    if len(results) > 1:
        for i, metric_name in enumerate(loaded_model.metrics_names[1:], start=1):
            print(f"{metric_name}: {results[i]}")

In [ ]:
print(f"Test Loss: {results}")

In [ ]:
import numpy as np
rmse = np.sqrt(0.00032631406793370843)
print(rmse)  # ≈ 0.01806

In [ ]:
import streamlit as st

# Assume rmse is your calculated RMSE value
rmse = 0.018064165298560253

# Display RMSE as a metric
st.metric(label="Model RMSE", value=f"{rmse:.4f}")

The LSTM forecasting model achieves an RMSE of 0.018 (≈1.8% error on normalized scale), indicating highly accurate predictions of hourly energy consumption. The forecast curve closely follows actual usage patterns, while red markers highlight anomalies that represent unusual consumption events. These anomalies may be linked to operational changes, equipment malfunctions, or abnormal demand surges, and should be reviewed for potential action.

In [ ]:
# After plotting your forecast and anomalies in Streamlit
st.write("""
**Data Interpretation:**
The LSTM forecasting model achieves an RMSE of **0.018** (≈1.8% error on normalized scale), indicating highly accurate predictions of hourly energy consumption.
The forecast curve closely follows actual usage patterns, while red markers highlight anomalies representing unusual consumption events.
These anomalies may be linked to operational changes, equipment malfunctions, or abnormal demand surges, and should be reviewed for potential action.
""")

## Conclusion

This project successfully developed and deployed an **LSTM (Long Short-Term Memory) neural network model** for energy consumption forecasting. The LSTM model achieved an RMSE of approximately **0.018** (about 1.8% error on normalized data), indicating high predictive accuracy in capturing hourly energy consumption patterns.

LSTM architecture was selected due to its ability to process **sequential, time-series data** and retain information over long time dependencies. This capability makes it ideal for modeling energy usage trends influenced by daily and seasonal cycles. In contrast to traditional machine learning models, LSTM handles non-linear relationships and complex temporal dependencies, improving both forecast accuracy and anomaly detection capabilities.

The Streamlit dashboard integrates live forecast visualization, anomaly indicators, and performance metrics to provide actionable insights for energy management. This approach enables efficient monitoring and early detection of abnormal consumption events, supporting proactive operational decisions.

Overall, the project demonstrates the effectiveness of **deep learning with LSTM** in delivering accurate energy consumption forecasts while supporting anomaly detection for improved operational intelligence.